In [1]:
import os
import sys

import semantic_kernel as sk

In [2]:

kernel = sk.Kernel()

In [3]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion(deployment,endpoint,api_key=api_key,api_version = "2023-07-01-preview"))

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
base_skills_directory = '../../../plugins'

In [7]:
custom_plugin = kernel.import_native_skill_from_directory(base_skills_directory , "APIPlugin")
writer_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "TranslatePlugin")

In [8]:
ask = """
Check the weather in Guangzhou and generate dressing tips based on the results into spanish and write them as travel weather reminder emails
"""
original_plan = await planner.create_plan_async(ask, kernel)

In [9]:
print(original_plan.generated_plan)

{
    "input": "Guangzhou weather",
    "subtasks": [
        {"function": "APIPlugin.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips", "args": {"language": "Spanish"}},
        {"function": "EmailPlugin.WeatherMail"}
    ]
}


In [10]:
await planner.execute_plan_async(original_plan, kernel)

'El clima en Guangzhou es muy caluroso, con una temperatura de 30 grados Celsius. Por lo tanto, es recomendable elegir prendas ligeras y transpirables como camisetas de algodón, blusas de seda o telas de lino. Estas telas permiten que el aire circule y te mantendrán fresco. Evita telas sintéticas que retienen el calor.\n\n2. Además, es aconsejable usar colores claros en tus prendas, ya que reflejan la luz solar y te mantendrán más fresco. Evita los colores oscuros que absorben el calor.\n\n3. Para mayor comodidad, viste con ropa holgada y suelta. Esto permitirá la circulación del aire alrededor de tu cuerpo y te ayudará a mantenerte fresco.\n\n4. No olvides protegerte del sol utilizando sombreros de ala ancha, gorras o viseras. Estos accesorios te ayudarán a proteger tu cabeza del sol y a mantenerla fresca.\n\n5. Es importante aplicar protector solar en tu piel expuesta para evitar quemaduras solares y daño causado por los rayos UV.\n\n6. En cuanto al calzado, es recomendable utilizar 